In [3]:
from functools import reduce
import pandas as pd
import glob
import os
import numpy as np

# Caminho da pasta com os arquivos CSV
pasta = './'  # ajuste se necessário

# Lista todos os arquivos CSV que começam com 'investing'
arquivos_csv = glob.glob(os.path.join(pasta, '*.csv'))

# Lê e concatena todos os arquivos
dfs = [pd.read_csv(f) for f in arquivos_csv]

# Converte a coluna de datas (ajuste o nome da coluna se necessário)

# Renomeia as colunas de fechamento e mínima em cada DataFrame e faz o merge por 'Data'
dfs_renomeados = []
nomes = ['DIV', 'HASH', 'IVV']

for df_atual, nome in zip(dfs, nomes):
    temp = df_atual[['Data', 'Último', 'Mínima']].copy()
    temp['Data'] = pd.to_datetime(temp['Data'], dayfirst=True, errors='coerce', format='%d.%m.%Y')
    temp = temp.rename(columns={
        'Último': f'close_{nome}',
        'Mínima': f'low_{nome}'
    })
    dfs_renomeados.append(temp)

# Merge dos três DataFrames pela coluna 'Data'
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Data', how='outer'), dfs_renomeados)

# Ordena por data crescente
df = df_merged.dropna().sort_values('Data', ascending=True).reset_index(drop=True)

# Converte as colunas de preços para float (substitui vírgula por ponto)
for nome in nomes:
    df[f'close_{nome}'] = df[f'close_{nome}'].str.replace(',', '.').astype(float)
    df[f'low_{nome}'] = df[f'low_{nome}'].str.replace(',', '.').astype(float)

for nome in nomes:
    df.sort_values('Data', ascending=True, inplace=True)
    df[f'r_{nome}_taxed'] = (df[f'close_{nome}'] / df[f'close_{nome}'].shift(1) - 1) * 0.85  # considerando 15% de imposto
    df[f'ra_{nome}_bh_taxed'] = df[f'r_{nome}_taxed'].cumsum()
    df[f'maxdown_{nome}'] = (df[f'low_{nome}'] / df[f'close_{nome}'].shift(1) - 1)
    #=SE(F4<0;SE(H4<-0,5%;H4-1%;-1%);SE(H4<-0,5%;H4+0,5%;-0,1%))        
    df[f'stop_{nome}'] = np.where(df[f'r_{nome}_taxed'].shift(1) < 0,
                                  np.where(df[f'maxdown_{nome}'].shift(1) < -0.005, df[f'maxdown_{nome}'].shift(1) - 0.01, -0.01),
                                  np.where(df[f'maxdown_{nome}'].shift(1) < -0.005, df[f'maxdown_{nome}'].shift(1) + 0.005, -0.001)
                                  )
    #=SE(H4<I4;I4*0,85-0,0575%-0,1%;F4)
    df[f'rstrat{nome}'] =np.where(df[f'maxdown_{nome}'] < df[f'stop_{nome}'],
                                  df[f'stop_{nome}'] * 0.85 - 0.000575 - 0.001, df[f'r_{nome}_taxed'])
    df[f'rstratacum{nome}'] = df[f'rstrat{nome}'].cumsum()

df = df.dropna().sort_values('Data', ascending=True)

# Salva em Excel
df.to_excel('análise.xlsx', index=False)
df.T

,1,2,3,4,5,6,7,8,9,10,...,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084
Data,2021-06-07 00:00:00,2021-06-08 00:00:00,2021-06-09 00:00:00,2021-06-10 00:00:00,2021-06-11 00:00:00,2021-06-14 00:00:00,2021-06-15 00:00:00,2021-06-16 00:00:00,2021-06-17 00:00:00,2021-06-18 00:00:00,...,2025-09-17 00:00:00,2025-09-18 00:00:00,2025-09-19 00:00:00,2025-09-22 00:00:00,2025-09-23 00:00:00,2025-09-24 00:00:00,2025-09-25 00:00:00,2025-09-26 00:00:00,2025-09-29 00:00:00,2025-09-30 00:00:00
close_DIV,74.41,73.85,73.67,73.81,73.99,73.62,73.54,72.8,72.58,72.47,...,106.95,106.62,106.66,106.53,107.61,107.57,106.45,106.76,107.03,107.49
low_DIV,74.0,73.83,73.67,73.64,72.5,73.5,73.3,72.8,72.47,72.29,...,105.59,106.24,106.6,105.71,106.5,107.36,106.3,106.35,107.03,107.0
close_HASH,34.9,32.4,34.6,34.45,34.72,36.52,36.39,36.0,34.0,32.35,...,89.27,90.9,88.84,85.9,84.93,86.41,83.5,83.73,86.91,86.4
low_HASH,34.62,30.16,33.13,34.18,34.31,36.32,36.36,34.74,33.92,31.98,...,88.03,89.95,88.76,85.86,84.67,85.74,82.62,82.86,84.93,85.53
close_IVV,231.46,231.47,232.0,234.1,237.4,234.9,232.9,232.8,230.2,229.4,...,392.93,396.15,398.05,401.42,394.91,396.5,397.79,398.25,398.0,399.8
low_IVV,230.01,229.64,231.12,231.7,234.25,232.5,232.12,229.64,228.7,226.23,...,388.53,393.72,396.2,397.32,394.39,395.03,393.14,396.0,396.36,396.0
r_DIV_taxed,0.00275,-0.006397,-0.002072,0.001615,0.002073,-0.004251,-0.000924,-0.008553,-0.002569,-0.001288,...,0.014468,-0.002623,0.000319,-0.001036,0.008617,-0.000316,-0.00885,0.002475,0.00215,0.003653
ra_DIV_bh_taxed,0.00275,-0.003647,-0.005718,-0.004103,-0.00203,-0.006281,-0.007204,-0.015758,-0.018326,-0.019614,...,0.35783,0.355207,0.355526,0.35449,0.363107,0.362791,0.353941,0.356416,0.358566,0.362219
maxdown_DIV,-0.002292,-0.007795,-0.002437,-0.000407,-0.017748,-0.006623,-0.004347,-0.010063,-0.004533,-0.003996,...,0.004089,-0.006639,-0.000188,-0.008907,-0.000282,-0.002323,-0.011806,-0.000939,0.002529,-0.00028


In [4]:
from functools import reduce
import pandas as pd
import glob
import os
import numpy as np
# # Desinstala as versões conflitantes
# %pip uninstall -y numpy pandas pmdarima arch
# # Instala o Numpy 1.x primeiro e fixa a versão
# %pip install "numpy>=1.26.4,<2.0.0" --no-cache-dir
# %pip install pandas==2.2.2 pmdarima==2.0.4 arch --no-cache-dir
# %pip show pmdarima scikit-learn numpy pandas arch
# %pip uninstall -y scikit-learn
# %pip install --no-cache-dir "scikit-learn==1.7.2"

import warnings
import pmdarima as pm
from arch import arch_model

# Configurações globais
MIN_OBS = 484  # mínimo de observações para rodar ARIMA-GARCH
_arima_order_cache = {}

def runArimaGarch(series_array: np.ndarray, nome: str = "default") -> float:
    # Sanitiza o histórico recebido do expanding()
    x = np.asarray(series_array, dtype="float64")
    x = x[np.isfinite(x)]  # remove nan/inf

    if x.size < MIN_OBS:
        return np.nan

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        # try:
        print(f"Processando {nome} com {x.size} observações...")
        if nome not in _arima_order_cache:
            probe = pm.auto_arima(
                x,
                d=0,
                max_p=3, max_q=3,
                seasonal=False,
                stepwise=True,
                suppress_warnings=True,
                error_action="ignore",
            )
            _arima_order_cache[nome] = probe.order  # (p,d,q)

        p, d, q = _arima_order_cache[nome]

        arima_fit = pm.ARIMA(order=(p, d, q)).fit(x)
        resid = arima_fit.resid()  # método existe no ARIMA do pmdarima [web:23]

        # (opcional) GARCH nos resíduos
        garch = arch_model(resid, vol="Garch", p=1, q=1, dist="normal", rescale=False)
        _ = garch.fit(disp="off", show_warning=False)

        # previsão 1 passo à frente da média (retorno esperado)
        return float(arima_fit.predict(n_periods=1)[0])  # API do pmdarima [web:16]

        # except Exception:
        #     return np.nan



# Caminho da pasta com os arquivos CSV
pasta = './'  # ajuste se necessário

# Lista todos os arquivos CSV que começam com 'investing'
arquivos_csv = glob.glob(os.path.join(pasta, '*.csv'))

# Lê e concatena todos os arquivos
dfs = [pd.read_csv(f) for f in arquivos_csv]

# Converte a coluna de datas (ajuste o nome da coluna se necessário)

# Renomeia as colunas de fechamento e mínima em cada DataFrame e faz o merge por 'Data'
dfs_renomeados = []
nomes = ['DIV', 'HASH', 'IVV']

for df_atual, nome in zip(dfs, nomes):
    temp = df_atual[['Data', 'Último', 'Mínima']].copy()
    temp['Data'] = pd.to_datetime(temp['Data'], dayfirst=True, errors='coerce', format='%d.%m.%Y')
    temp = temp.rename(columns={
        'Último': f'close_{nome}',
        'Mínima': f'low_{nome}'
    })
    dfs_renomeados.append(temp)

# Merge dos três DataFrames pela coluna 'Data'
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Data', how='outer'), dfs_renomeados)

# Ordena por data crescente
df = df_merged.dropna().sort_values('Data', ascending=True).reset_index(drop=True)

# Converte as colunas de preços para float (substitui vírgula por ponto)
for nome in nomes:
    df[f'close_{nome}'] = df[f'close_{nome}'].str.replace(',', '.').astype(float)
    df[f'low_{nome}'] = df[f'low_{nome}'].str.replace(',', '.').astype(float)

for nome in nomes:
    df.sort_values('Data', ascending=True, inplace=True)
    df[f'r_{nome}_taxed'] = (df[f'close_{nome}'] / df[f'close_{nome}'].shift(1) - 1) * 0.85  # considerando 15% de imposto
    df[f'lnr_{nome}_taxed'] = np.log1p(df[f'r_{nome}_taxed'])

    print(f'Calculando ARIMA-GARCH para {nome}...')
    df = df[np.isfinite(df[f'lnr_{nome}_taxed'])].copy()  # remove nan/inf do ativo
    _arima_order_cache.clear()  # ok por ativo
    fn = lambda arr, n=nome: runArimaGarch(arr, nome=n)
    # print(df)
    df[f'rpred{nome}'] = (
                            df[f'lnr_{nome}_taxed']
                            .rolling(window=MIN_OBS, min_periods=MIN_OBS)
                            .apply(lambda x: runArimaGarch(x, nome), raw=True)
                        )

df = df.dropna().sort_values('Data', ascending=True)

# Salva em Excel
df[['Data', 'r_DIV_taxed', 'r_HASH_taxed', 'r_IVV_taxed', 'rpredDIV', 'rpredHASH', 'rpredIVV']].to_excel('análise3.xlsx', index=False)
df.T

Calculando ARIMA-GARCH para DIV...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 observações...
Processando DIV com 484 obser

,486,487,488,489,490,491,492,493,494,495,...,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084
Data,2023-05-16 00:00:00,2023-05-17 00:00:00,2023-05-18 00:00:00,2023-05-19 00:00:00,2023-05-22 00:00:00,2023-05-23 00:00:00,2023-05-24 00:00:00,2023-05-25 00:00:00,2023-05-26 00:00:00,2023-05-29 00:00:00,...,2025-09-17 00:00:00,2025-09-18 00:00:00,2025-09-19 00:00:00,2025-09-22 00:00:00,2025-09-23 00:00:00,2025-09-24 00:00:00,2025-09-25 00:00:00,2025-09-26 00:00:00,2025-09-29 00:00:00,2025-09-30 00:00:00
close_DIV,73.14,73.65,74.14,74.55,74.57,74.42,73.79,74.19,74.65,74.61,...,106.95,106.62,106.66,106.53,107.61,107.57,106.45,106.76,107.03,107.49
low_DIV,73.1,73.57,73.4,74.01,71.65,74.09,73.62,74.0,74.19,74.46,...,105.59,106.24,106.6,105.71,106.5,107.36,106.3,106.35,107.03,107.0
close_HASH,22.73,23.0,22.6,22.79,22.35,22.9,22.49,22.65,22.79,23.0,...,89.27,90.9,88.84,85.9,84.93,86.41,83.5,83.73,86.91,86.4
low_HASH,22.35,22.36,22.36,22.62,22.35,22.71,22.14,22.4,22.61,23.0,...,88.03,89.95,88.76,85.86,84.67,85.74,82.62,82.86,84.93,85.53
close_IVV,224.5,226.7,230.53,231.49,230.3,227.8,225.32,231.0,231.6,233.39,...,392.93,396.15,398.05,401.42,394.91,396.5,397.79,398.25,398.0,399.8
low_IVV,222.59,224.52,227.03,230.0,229.27,227.28,224.18,227.46,230.01,231.01,...,388.53,393.72,396.2,397.32,394.39,395.03,393.14,396.0,396.36,396.0
r_DIV_taxed,-0.006802,0.005927,0.005655,0.004701,0.000228,-0.00171,-0.007196,0.004608,0.00527,-0.000455,...,0.014468,-0.002623,0.000319,-0.001036,0.008617,-0.000316,-0.00885,0.002475,0.00215,0.003653
lnr_DIV_taxed,-0.006825,0.005909,0.005639,0.00469,0.000228,-0.001711,-0.007222,0.004597,0.005256,-0.000456,...,0.014365,-0.002626,0.000319,-0.001037,0.00858,-0.000316,-0.008889,0.002472,0.002147,0.003647
rpredDIV,-0.000008,0.000009,0.000017,0.000023,0.000032,0.00003,0.000033,0.000048,0.000061,0.000053,...,0.000517,0.000507,0.000525,0.000511,0.000541,0.000563,0.000541,0.000557,0.000565,0.000563
